In [115]:
import pandas as pd

df = pd.read_csv('game_data.csv')

df = df.drop(columns=['scrape_id', 'main_story_polled', 'main_+_extras_polled', 'completionist_polled', 'all_playstyles_polled', 'developers', 'publishers', 'aliases', 'alias'])

df = df.dropna(subset=['genres'])
df = df.dropna(subset=['platforms'])

df['rating'] = pd.to_numeric(df['rating'], errors="coerce")
df = df.dropna(subset=['rating'])
df['rating'] = df['rating'] * 100
df['rating'] = df['rating'].astype(int)


df = df.dropna(subset=['na', 'eu', 'jp'], how='all')
df['eu'] = df['eu'].fillna(df['na'])
df['jp'] = df['jp'].fillna(df['na'])
df['eu'] = df['eu'].fillna(df['jp'])
df['jp'] = df['jp'].fillna(df['eu'])
df['na'] = df['na'].fillna(df['eu'])
df['eu'] = pd.to_datetime(df['eu'])
df['na'] = pd.to_datetime(df['na'])
df['jp'] = pd.to_datetime(df['jp'])

df = df.dropna(subset=['developer', 'publisher'], how='all')
df['developer'].fillna(df['publisher'], inplace=True)
df['publisher'].fillna(df['developer'], inplace=True)

for col in df.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        if "median" not in col:
            df = df.drop(columns=[col])
        else:
            df[col] = pd.to_timedelta(df[col], errors="coerce")

for col in df.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        df[col].fillna(df[col].mean(), inplace=True)

missing_values = df.isnull().sum()
print(missing_values)
df.count
# len(df.columns)


name                     0
rating                   0
beat_count               0
platforms                0
genres                   0
developer                0
publisher                0
na                       0
main_story_median        0
main_+_extras_median     0
completionist_median     0
all_playstyles_median    0
eu                       0
jp                       0
dtype: int64


<bound method DataFrame.count of                                              name  rating  beat_count  \
0                                   'Splosion Man      75         152   
1                     .hack//G.U. Vol. 1: Rebirth      76         225   
2                   .hack//G.U. Vol. 2: Reminisce      79         135   
3                  .hack//G.U. Vol. 3: Redemption      78         106   
4                                .hack//Infection      73         308   
...                                           ...     ...         ...   
10776          The Dark Pictures: The Devil in Me      66         233   
10777                Star Ocean: The Divine Force      76         103   
10778  Voice of Cards: The Isle Dragon Roars Demo      72          35   
10779                       Dogs Organized Neatly      76          43   
10780                                  Bloodshore      61          41   

                                               platforms  \
0                             

In [116]:
df['developer_count'] = df.groupby('developer')['name'].transform('count')
df['publisher_count'] = df.groupby('publisher')['name'].transform('count')

from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Vytvoříme 1D pole s počtem her vyvinutých každým vývojářem
X = np.array(df['developer_count']).reshape(-1, 1)

# Použijeme metodu lokte (elbow method) k určení optimálního počtu skupin
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()
#
# # Podle výsledků grafu vybereme optimální počet skupin (v našem případě 3)
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
df['developer_size'] = kmeans.fit_predict(X)

X = np.array(df['publisher_count']).reshape(-1, 1)

# Použijeme metodu lokte (elbow method) k určení optimálního počtu skupin
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
# plt.plot(range(1, 11), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# # Podle výsledků grafu vybereme optimální počet skupin (v našem případě 3)
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
df['publisher_size'] = kmeans.fit_predict(X)


# Definujte hranice pro rozdělení
threshold1 = 33.33  # První hranice
threshold2 = 66.66  # Druhá hranice

# Vytvořte funkci pro přiřazení do skupiny
def assign_group(value):
    if value <= threshold1:
        return 0
    elif value <= threshold2:
        return 1
    else:
        return 2

# Aplikujte funkci na hodnocení hry
df['rating_group'] = df['rating'].apply(assign_group)

df = df.drop(columns=["developer","rating", "publisher", "name", "developer_count", "publisher_count", "all_playstyles_median"])
df

,beat_count,platforms,genres,na,main_story_median,main_+_extras_median,completionist_median,eu,jp,developer_size,publisher_size,rating_group
0,152,['Xbox 360'],['Action'],2009-07-22,0 days 08:00:00,0 days 10:00:00,0 days 16:28:00,2009-07-22,2009-07-22,0,0,2
1,225,"['Nintendo Switch', 'PC', 'PlayStation 2', 'Pl...","['Action', 'Role-Playing']",2006-10-24,0 days 18:49:00,1 days 00:00:00,1 days 12:56:00,2006-10-24,2006-05-18,0,0,2
2,135,"['PC', 'PlayStation 2', 'PlayStation 4']","['Action', 'Role-Playing']",2007-05-08,0 days 20:13:00,1 days 06:00:00,1 days 23:26:00,2007-05-08,2006-09-28,0,0,2
3,106,"['PC', 'PlayStation 2', 'PlayStation 4']","['Action', 'Role-Playing']",2008-03-25,0 days 21:00:00,1 days 07:49:00,2 days 02:04:00,2008-03-25,2007-01-18,0,0,2
4,308,['PlayStation 2'],"['Action', 'role-playing']",2003-02-11,0 days 17:05:00,0 days 20:00:00,1 days 06:00:00,2003-02-11,2003-02-11,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
10776,233,"['PC', 'PlayStation 4', 'PlayStation 5', 'Xbox...","['Third-Person', 'Adventure', 'Horror']",2022-11-18,0 days 07:00:00,0 days 08:34:00,0 days 12:35:00,2022-11-18,2022-11-18,0,0,1
10777,103,"['PC', 'PlayStation 4', 'PlayStation 5', 'Xbox...","['Third-Person', 'Real-Time', 'Action', 'Role-...",2022-10-27,1 days 07:30:00,1 days 22:00:00,2 days 22:00:00,2022-10-27,2022-10-27,0,1,2
10778,35,"['Nintendo Switch', 'PC', 'PlayStation 4']",['Card Game'],2021-09-24,0 days 01:10:00,0 days 01:40:00,0 days 01:30:00,2021-09-24,2021-09-24,2,1,2
10779,43,['PC'],['Puzzle'],2021-11-01,0 days 02:00:00,0 days 02:05:00,0 days 02:30:00,2021-11-01,2021-11-01,0,0,2


In [117]:
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler

df["genres"] = df["genres"].apply(literal_eval)
df["platforms"] = df["platforms"].apply(literal_eval)

df['is_PC'] = df['platforms'].apply(lambda x: 1 if 'PC' in x else 0)
df['is_Console'] = df['platforms'].apply(lambda x: 1 if 'PC' not in x else 0)


# Rozbití listů na samostatné řádky
exploded_genres = df['genres'].explode()

# Výpočet četností jednotlivých žánrů
genre_counts = exploded_genres.value_counts()
rare_genres = genre_counts[genre_counts < 50].index

mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(df['genres']), columns=mlb.classes_, index=df.index)

# mlb = MultiLabelBinarizer()
# platforms_encoded = pd.DataFrame(mlb.fit_transform(df['platforms']), columns=mlb.classes_, index=df.index)

data_encoded = pd.concat([df , genres_encoded], axis=1)

data_encoded['na_year'] = data_encoded['na'].dt.year
data_encoded['na_month'] = data_encoded['na'].dt.month
data_encoded['na_day'] = data_encoded['na'].dt.day

# data_encoded['eu_year'] = data_encoded['eu'].dt.year
# data_encoded['eu_month'] = data_encoded['eu'].dt.month
# data_encoded['eu_day'] = data_encoded['eu'].dt.day
#
# data_encoded['jp_year'] = data_encoded['jp'].dt.year
# data_encoded['jp_month'] = data_encoded['jp'].dt.month
# data_encoded['jp_day'] = data_encoded['jp'].dt.day


data_encoded = data_encoded.drop(columns=["genres", "platforms", "na", "jp", "eu"])

for col in data_encoded.columns:
    if "main_story" in col or "main_+_extras" in col or "completionist" in col or "all_playstyles" in col:
        data_encoded[col] = data_encoded[col].dt.total_seconds() / 60

columns_to_check = ['main_story_median', 'main_+_extras_median', 'completionist_median']
threshold = 3  # Práh pro identifikaci odlehlých hodnot

# Vytvoření kopie DataFrame pro filtrování
filtered_data = data_encoded.copy()

# Iterace přes vybrané sloupce
for column in columns_to_check:
    # Výpočet z-score
    z_scores = np.abs((data_encoded[column] - data_encoded[column].mean()) / data_encoded[column].std())

    # Filtrace odlehlých hodnot
    filtered_data = filtered_data[z_scores < threshold]

data_encoded = filtered_data

scaler = MinMaxScaler()

data_encoded['main_story_normalized'] = scaler.fit_transform(data_encoded[['main_story_median']])
data_encoded['main_+_extras_normalized'] = scaler.fit_transform(data_encoded[['main_+_extras_median']])
data_encoded['completionist_normalized'] = scaler.fit_transform(data_encoded[['completionist_median']])
data_encoded['beat_count'] = scaler.fit_transform(data_encoded[['beat_count']])
# data_encoded[['main_story_median', 'main_+_extras_median', 'completionist_median']] = StandardScaler().fit_transform(data_encoded[['main_story_median', 'main_+_extras_median', 'completionist_median']])

# data_encoded[['beat_count']] = StandardScaler().fit_transform(data_encoded[['beat_count']])

data_encoded = data_encoded.drop(columns=[*columns_to_check, *rare_genres])

data_encoded

C:\Users\Vincent\AppData\Local\Temp\ipykernel_17600\271219847.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[z_scores < threshold]
C:\Users\Vincent\AppData\Local\Temp\ipykernel_17600\271219847.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[z_scores < threshold]


,beat_count,developer_size,publisher_size,rating_group,is_PC,is_Console,Action,Adventure,Arcade,Beat em Up,...,Tower Defense,Turn-Based,Virtual Reality,Visual Novel,na_year,na_month,na_day,main_story_normalized,main_+_extras_normalized,completionist_normalized
0,0.004609,0,0,2,0,1,1,0,0,0,...,0,0,0,0,2009,7,22,0.078904,0.041655,0.030806
1,0.007367,0,0,2,1,0,1,0,0,0,...,0,0,0,0,2006,10,24,0.186035,0.100070,0.069134
2,0.003967,0,0,2,1,0,1,0,0,0,...,0,0,0,0,2007,5,8,0.199901,0.125104,0.088798
3,0.002871,0,0,2,1,0,1,0,0,0,...,0,0,0,0,2008,3,25,0.207659,0.132684,0.093730
4,0.010502,0,0,2,0,1,1,0,0,0,...,0,0,0,0,2003,2,11,0.168868,0.083380,0.056150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776,0.007669,0,0,1,1,0,0,1,0,0,...,0,0,0,0,2022,11,18,0.069000,0.035675,0.023534
10777,0.002758,0,1,2,1,0,1,0,0,0,...,0,0,0,0,2022,10,27,0.311654,0.191864,0.131059
10778,0.000189,2,1,2,1,0,0,0,0,0,...,0,0,0,0,2021,9,24,0.011225,0.006885,0.002778
10779,0.000491,0,0,2,1,0,0,0,0,0,...,0,0,0,0,2021,11,1,0.019478,0.008623,0.004651


In [118]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, f1_score

# Najde duplicitní sloupce
duplicate_column_names = data_encoded.columns[data_encoded.columns.duplicated()]
# Odstraní duplicitní sloupce
data_encoded = data_encoded.drop(columns=duplicate_column_names)

X = data_encoded.drop('rating_group', axis=1)
y = data_encoded['rating_group']


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Rozdělení dat na trénovací a testovací set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Definujeme parametry pro Grid Search
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}


# Inicializujte model
gb = GradientBoostingClassifier()

# Definujte Randomized Search
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=3, scoring='f1_macro', n_jobs=18)
# Trénování modelu pomocí Randomized Search
grid_search.fit(X_train, y_train)

# Získáme nejlepší parametry
print("Nejlepší parametry: ", grid_search.best_params_)

# Získáme nejlepší model z Grid Search
best_model = grid_search.best_estimator_
# Použijeme tento model k predikci na testovacích datech
y_pred = best_model.predict(X_test)

# Výpočet F-skóre
f_score = f1_score(y_test, y_pred, average='macro')

print("F-skóre na testovacích datech: ", f_score)

# best_model.feature_importances_

KeyboardInterrupt: 